In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64

import time
from datetime import datetime


# Scikit-Learn Model Deployment Use Case

In this case, we will build a very large ensemble model (here, Random Foreast with 512 trees) on a digits dataset  (not very original !!!) and generate a SQL code for deployment using the web service. 

We then execute the SQL code on a local database (postgresql) and compare the SQL execution result with scikit-learn predict/predict_proba/.predict_log_proba result. 

Both results are stored in pandas dataframes. 


## Build a scikit-learn model

In [2]:
from sklearn import datasets

digits = datasets.load_digits()
X = digits.data
n_classes = len(digits.target_names)

In [3]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=32, max_depth=7, min_samples_leaf=30, random_state = 1960)
clf.fit(digits.data, digits.target)

RandomForestClassifier(max_depth=7, min_samples_leaf=30, n_estimators=32,
                       random_state=1960)

In [4]:
#clf.__dict__

## Generate SQL Code from the Model

In [5]:


def test_ws_sql_gen(pickle_data):
    WS_URL="http://localhost:1888/model"
    b64_data = base64.b64encode(pickle_data).decode('utf-8')
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"duckdb"}
    r = requests.post(WS_URL, json=data)
    #print(r.__dict__)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;



In [6]:
pickle_data = pickle.dumps(clf)
lSQL1 = test_ws_sql_gen(pickle_data)
(lTempSQLs , lFinalSQL1 , lFinalSQL2) = lSQL1
lSQLs = []
for t in lTempSQLs:
    (name, lCreateTableSQL , lCreateIndexSQL, lInsertSQL) = t
    lSQLs = lSQLs + [lCreateTableSQL , lCreateIndexSQL, lInsertSQL] 
lSQLs = lSQLs + [lFinalSQL1]
lSQLs = [t for t in lSQLs if t is not None]
lSQLs = [t.replace("DOUBLE_PRECISION", "DOUBLE") for t in lSQLs]

with open('data/digits_64_features_RF_32_DuckDB_with_materialization.sql', 'w') as f:
    for s in lSQLs:
        print("XXXXXXXXXXXXXXXXXXXXXXXXX" , s[:50] , s[-50:])
        f.write(s + ";\n\n")

for lSQL in lSQLs:
    N = len(lSQL)
    P = 60
    print(lSQL[0:P] + "..." + lSQL[N//2:(N//2 + P)] + "..." + lSQL[-P:])

XXXXXXXXXXXXXXXXXXXXXXXXX 
CREATE TEMPORARY TABLE "TMP_20221025001409_VSCWP7  
	"LogProba_9" DOUBLE
)

 ON COMMIT PRESERVE ROWS
XXXXXXXXXXXXXXXXXXXXXXXXX WITH "DT_node_lookup" AS 
(SELECT "ADS"."KEY" AS " OM "RF_Model_9") AS "RF_esu_0") AS "RF_B0") AS "T"
XXXXXXXXXXXXXXXXXXXXXXXXX 
CREATE TEMPORARY TABLE "TMP_20221025001409_RIIIWV  
	"LogProba_9" DOUBLE
)

 ON COMMIT PRESERVE ROWS
XXXXXXXXXXXXXXXXXXXXXXXXX WITH "DT_node_lookup_10" AS 
(SELECT "ADS"."KEY" A M "RF_Model_19") AS "RF_esu_1") AS "RF_B1") AS "T"
XXXXXXXXXXXXXXXXXXXXXXXXX 
CREATE TEMPORARY TABLE "TMP_20221025001409_2EZ2I9  
	"LogProba_9" DOUBLE
)

 ON COMMIT PRESERVE ROWS
XXXXXXXXXXXXXXXXXXXXXXXXX WITH "DT_node_lookup_20" AS 
(SELECT "ADS"."KEY" A M "RF_Model_29") AS "RF_esu_2") AS "RF_B2") AS "T"
XXXXXXXXXXXXXXXXXXXXXXXXX 
CREATE TEMPORARY TABLE "TMP_20221025001409_DZBJED  
	"LogProba_9" DOUBLE
)

 ON COMMIT PRESERVE ROWS
XXXXXXXXXXXXXXXXXXXXXXXXX WITH "DT_node_lookup_30" AS 
(SELECT "ADS"."KEY" A M "RF_Model_31") AS "RF_esu

## Execute the SQL Code

In [7]:
# save the dataset in a database table

conf = {"memory_limit": "1000mb", "temp_directory" : "", "threads" : 1, "external_threads" : 0}
engine = sa.create_engine("duckdb:///:memory:" , connect_args={"config": conf}, echo=False)
conn = engine.connect()
# conn.execute("PRAGMA threads=1")
# conn.execute("PRAGMA external_threads=1")

lTable = pd.DataFrame(digits.data);
lTable.columns = ['Feature_' + str(c) for c in range(digits.data.shape[1])]
lTable['KEY'] = range(lTable.shape[0])
lTable.to_csv("data/digits.csv", index=False)
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)

print("DUCKDB_VERSION", [x for x in conn.execute("PRAGMA version")])

DUCKDB_VERSION [('0.5.1', '7c111322d')]


In [8]:
# create temp tables
for lSQL in lSQLs[:-1]:
    conn.exec_driver_sql(lSQL)
sql_output = pd.read_sql(lSQLs[-1] , conn);

sql_output = sql_output.sort_values(by='KEY').reset_index(drop=True)
conn.close()

In [9]:
sql_output.sample(12, random_state=1960)

,KEY,Score_0,Score_1,Score_2,Score_3,Score_4,Score_5,Score_6,Score_7,Score_8,...,LogProba_2,LogProba_3,LogProba_4,LogProba_5,LogProba_6,LogProba_7,LogProba_8,LogProba_9,Decision,DecisionProba
1197,1197,None,None,None,None,None,None,None,None,None,...,-2.664286,-2.382074,-3.492821,-1.771092,-3.682391,-2.012006,-1.448500,-2.394452,8,0.234922
913,913,None,None,None,None,None,None,None,None,None,...,-2.245592,-2.277992,-4.218420,-3.296202,-3.087859,-3.812117,-0.677330,-2.427624,8,0.507971
893,893,None,None,None,None,None,None,None,None,None,...,-2.769139,-2.870889,-3.816887,-0.647482,-5.654529,-2.467505,-2.087442,-3.222668,5,0.523362
604,604,None,None,None,None,None,None,None,None,None,...,-3.077985,-4.469638,-1.939647,-1.518858,-1.429768,-3.630461,-2.180310,-4.761802,6,0.239364
743,743,None,None,None,None,None,None,None,None,None,...,-5.033573,-7.560080,-0.658430,-4.269875,-4.630090,-1.275258,-3.623043,-3.180901,4,0.517664
556,556,None,None,None,None,None,None,None,None,None,...,-1.898130,-2.894140,-3.374349,-4.947656,-3.076462,-3.893662,-0.847356,-3.264704,8,0.428547
664,664,None,None,None,None,None,None,None,None,None,...,-2.574034,-2.347899,-3.267876,-3.138343,-3.080330,-3.546194,-0.849352,-2.357946,8,0.427692
195,195,None,None,None,None,None,None,None,None,None,...,-3.020168,-7.987524,-3.417120,-4.103369,-0.250398,-4.832728,-3.240598,-6.139885,6,0.778491
692,692,None,None,None,None,None,None,None,None,None,...,-4.273311,-3.991354,-3.413860,-0.311865,-4.095399,-3.192069,-2.367138,-4.058217,5,0.732081
1589,1589,None,None,None,None,None,None,None,None,None,...,-3.942291,-4.438870,-3.234805,-0.710680,-2.823992,-2.524735,-2.220892,-4.106573,5,0.491310


In [10]:
sql_output.Decision.value_counts()

7    199
1    186
9    183
0    181
4    178
6    177
2    176
3    175
5    174
8    168
Name: Decision, dtype: int64

## Scikit-learn Prediction

In [11]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_score = pd.DataFrame(columns=['Score_' + str(c) for c in range(n_classes)]);
skl_output_proba = pd.DataFrame(clf.predict_proba(X), columns=['Proba_' + str(c) for c in range(n_classes)])
skl_output_log_proba = pd.DataFrame(clf.predict_log_proba(X), columns=['LogProba_' + str(c) for c in range(n_classes)])
skl_output_decision = pd.DataFrame(clf.predict(X), columns=['Decision'])
skl_output = pd.concat([skl_output_key, skl_output_score, skl_output_proba, skl_output_log_proba, skl_output_decision] , axis=1)
skl_output.sample(12, random_state=1960)


/usr/lib/python3/dist-packages/sklearn/ensemble/_forest.py:922: RuntimeWarning: divide by zero encountered in log
  return np.log(proba)


,KEY,Score_0,Score_1,Score_2,Score_3,Score_4,Score_5,Score_6,Score_7,Score_8,...,LogProba_1,LogProba_2,LogProba_3,LogProba_4,LogProba_5,LogProba_6,LogProba_7,LogProba_8,LogProba_9,Decision
1197,1197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.088894,-2.664286,-2.382074,-3.492821,-1.771092,-3.682391,-2.012006,-1.448500,-2.394452,8
913,913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.777863,-2.245592,-2.277992,-4.218420,-3.296202,-3.087859,-3.812117,-0.677330,-2.427624,8
893,893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.527817,-2.769139,-2.870889,-3.816887,-0.647482,-5.654529,-2.467505,-2.087442,-3.222668,5
604,604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.244274,-3.077985,-4.469638,-1.939647,-1.518858,-1.429768,-3.630461,-2.180310,-4.761802,6
743,743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.109474,-5.033573,-7.560080,-0.658430,-4.269875,-4.630090,-1.275258,-3.623043,-3.180901,4
556,556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.607941,-1.898130,-2.894140,-3.374349,-4.947656,-3.076462,-3.893662,-0.847356,-3.264704,8
664,664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.951315,-2.574034,-2.347899,-3.267876,-3.138343,-3.080330,-3.546194,-0.849352,-2.357946,8
195,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.734942,-3.020168,-7.987524,-3.417120,-4.103369,-0.250398,-4.832728,-3.240598,-6.139885,6
692,692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.471108,-4.273311,-3.991354,-3.413860,-0.311865,-4.095399,-3.192069,-2.367138,-4.058217,5
1589,1589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.144889,-3.942291,-4.438870,-3.234805,-0.710680,-2.823992,-2.524735,-2.220892,-4.106573,5


## Comparing the SQL and Scikit-learn Predictions

In [12]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [13]:
sql_skl_join.sample(12, random_state=1960)

,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Score_3_skl,Score_4_skl,Score_5_skl,Score_6_skl,Score_7_skl,Score_8_skl,...,LogProba_2_sql,LogProba_3_sql,LogProba_4_sql,LogProba_5_sql,LogProba_6_sql,LogProba_7_sql,LogProba_8_sql,LogProba_9_sql,Decision_sql,DecisionProba
1197,1197,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.664286,-2.382074,-3.492821,-1.771092,-3.682391,-2.012006,-1.448500,-2.394452,8,0.234922
913,913,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.245592,-2.277992,-4.218420,-3.296202,-3.087859,-3.812117,-0.677330,-2.427624,8,0.507971
893,893,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.769139,-2.870889,-3.816887,-0.647482,-5.654529,-2.467505,-2.087442,-3.222668,5,0.523362
604,604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.077985,-4.469638,-1.939647,-1.518858,-1.429768,-3.630461,-2.180310,-4.761802,6,0.239364
743,743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5.033573,-7.560080,-0.658430,-4.269875,-4.630090,-1.275258,-3.623043,-3.180901,4,0.517664
556,556,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-1.898130,-2.894140,-3.374349,-4.947656,-3.076462,-3.893662,-0.847356,-3.264704,8,0.428547
664,664,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2.574034,-2.347899,-3.267876,-3.138343,-3.080330,-3.546194,-0.849352,-2.357946,8,0.427692
195,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.020168,-7.987524,-3.417120,-4.103369,-0.250398,-4.832728,-3.240598,-6.139885,6,0.778491
692,692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4.273311,-3.991354,-3.413860,-0.311865,-4.095399,-3.192069,-2.367138,-4.058217,5,0.732081
1589,1589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.942291,-4.438870,-3.234805,-0.710680,-2.823992,-2.524735,-2.220892,-4.106573,5,0.491310


In [14]:
condition = (sql_skl_join.Decision_sql != sql_skl_join.Decision_skl)
sql_skl_join[condition]


,KEY_skl,Score_0_skl,Score_1_skl,Score_2_skl,Score_3_skl,Score_4_skl,Score_5_skl,Score_6_skl,Score_7_skl,Score_8_skl,...,LogProba_2_sql,LogProba_3_sql,LogProba_4_sql,LogProba_5_sql,LogProba_6_sql,LogProba_7_sql,LogProba_8_sql,LogProba_9_sql,Decision_sql,DecisionProba
